# Creating and exploring the OWL/RL + SWRL Ontologies using CogniPy by Cognitum Services S.A.

## Setting up the environment
Initialization of the Ontology editor in Jupyter Notebook

In [2]:
from cognipy.ontology import Ontology #the ontology processing class
from cognipy.ontology import CQL #SPARQL format tailored for Contolled Natural Language
from cognipy.edit import CnlEditBox #The pywidget based ontology editor (offspirng of the Fluent Editor)
from cognipy.edit import CnlQueryForConcept #The pywidget based ontology quaty widget 
from cognipy.ontology import encode_string_for_graph_label #complex datatypes encoder for the graph labels in graph visualisation
import textwrap

def graph_attribute_formatter(val):
    if isinstance(val,list) or isinstance(val,set):
        return " | ".join(list(map(lambda i:encode_string_for_graph_label(graph_attribute_formatter(i)),val)))
    elif isinstance(val,dict):
        return " | ".join(list(map(lambda i:i[0]+" : "+encode_string_for_graph_label(graph_attribute_formatter(i[1])),val.items())))
    else:
        return encode_string_for_graph_label(textwrap.fill(str(val),40))

# 'African Wildlife'.
Loading the editor for the basic "safari" ontology. The Ontology is inspired by 'A Semantic Web Primer.' by 'Antoniou, G, van Harmelen, F.' 'MIT Press, 2003.' 'http://www.csd.uoc.gr/~hy566/SWbook.pdf', tuned to OWL/RL+SWRL profile.

### Part-1: 'simple hierarchy of beings'. 

We will have many editors here and we will store all of them in the dict

In [3]:
parts_editors={}

Lets setup the fist ontology editor for the general knowledge. The editor allows you to enter the knowledge in text format, and support you with the prdictive editor.

In [4]:
parts_editors['01']=CnlEditBox('part_01.encnl')
parts_editors['01']

<IPython.core.display.Javascript object>

Lets define the function that will build the ontology based on the dict of ontology editors and the optional additional editor for testing purposes. All it is doing it is concatenaiting the values of all the editors and puts them into the newly created ontology object.

In [5]:
def buildOntology(parts_editors,test_editor=None):
    return Ontology("cnl/string",
                    "\n\n".join(k.getvalue() for k in parts_editors.values())+('' if test_editor is None else test_editor.getvalue()), 
                    evaluator = lambda e:eval(e,globals(),locals()), 
                    graph_attribute_formatter = graph_attribute_formatter)

The ontology object alows you to draw the materialised graph using several layout algorythms. Lets draw our base ontology.

In [7]:
onto=buildOntology(parts_editors)
onto.draw_graph(layout='hierarchical')

OSError: [Errno 22] Invalid argument

It is fully compatible with OWL/RDF format (via OWLAPI). We can always export it as it is.

In [ ]:
print(onto.as_rdf())

### Part-2: Disjointness
In OWL/RL+SWRL profile we deal with Open World Assumption. It means that we need to explicitly specify all the objects that are different from each other. We cannot assume (like e.g. object oriented programming language do) that the different names mean the different things, therefore we need to specify explicitly if two things are different.

In [ ]:
parts_editors['02']=CnlEditBox('part_02.encnl',onto)
parts_editors['02']

Lets test how it works on the example. The example will be defining several instances, that aim to break the consistency of the ontology in som not obvious way.

In [ ]:
test_01_edit = CnlEditBox('test_01.encnl',onto)
test_01_edit

Lets load all the parts ontology ontology togheter with the testing part into the newly created obeject. 

In [ ]:
onto=buildOntology(parts_editors,test_01_edit)

We will need also some way to describe what is wrong `printReasoningInfo` and why `printWhy`.

In [ ]:
import json

def printReasoningInfo(onto):
    info=onto.reasoningInfo()
    if info == "":
        print('all good!')
    else:
        print(json.dumps(json.loads(info), indent=2, sort_keys=True))

def printWhy(onto,fact):
    info=json.loads(onto.why(fact))
    print(json.dumps(info, indent=2, sort_keys=True))

So what is wrong here?

In [ ]:
printReasoningInfo(onto)

Lets draw it on a diagram, this time using force directed layout.

In [ ]:
onto.draw_graph(layout='force directed')

In [ ]:
printWhy(onto,"Mary-01 is a animal?")

### Part-3: Modal expressions and part-whole relationships.

In [ ]:
parts_editors['03']=CnlEditBox('part_03.encnl',onto)
parts_editors['03']

In [ ]:
test_02_edit = CnlEditBox('test_02.encnl',onto)
test_02_edit

In [ ]:
onto=buildOntology(parts_editors,test_02_edit)

In [ ]:
printReasoningInfo(onto)

In [ ]:
onto.draw_graph(layout="force directed")

In [ ]:
q1_edit = CnlQueryForConcept('query_01.encnl',onto=onto)
q1_edit

In [ ]:
onto.select_instances_of(q1_edit.getvalue())

In [ ]:
print(onto.why("Leo-01 is an animal?"))

In [ ]:
onto.sparql_query(CQL("""select ?a1 ?a2 {
                            ?a1 rdf:type <animal>. 
                            ?a2 rdf:type <animal>. 
                            ?a1 <eats> ?a2. 
                        }""","http://cognitum.eu/african_wildlife#"))